In [1]:
import os
import ee
import pickle
import geemap
import geemap.colormaps as cm

# ee.Authenticate()

In [2]:
try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


In [3]:
# File name where the dictionary is stored
pickle_file = '../data/areas_of_interest.pickle'

# Initialize the areas_of_interest dictionary
if os.path.exists(pickle_file):
    # If the pickle file exists, load the dictionary from the file
    with open(pickle_file, 'rb') as handle:
        areas_of_interest = pickle.load(handle)
    print("Loaded areas of interest from pickle file.")
else:
    # If the pickle file does not exist, initialize an empty dictionary
    areas_of_interest = {}
    print("Initialized an empty dictionary for areas of interest.")

Loaded areas of interest from pickle file.


In [14]:
# Map = geemap.Map()
# Map

In [5]:
if Map.draw_last_feature is None:
    print("No new feature drawn. Nothing to retrieve.")
else:
    # Code for selecting the area
    fc = ee.FeatureCollection(Map.draw_last_feature)
    region = fc.geometry()
    coords = region.getInfo()['coordinates']
    area_name = input("Enter a name for this area: ")

    # Check if the area name already exists in the dictionary
    if area_name in areas_of_interest:
        print(f"The area name '{area_name}' already exists. Please choose a different name.")
    else:
        # Save the coordinates with area_name as ID
        areas_of_interest[area_name] = coords
        print(f"Area '{area_name}' saved successfully.")

        # Pickling the updated dictionary
        with open('areas_of_interest.pickle', 'wb') as handle:
            pickle.dump(areas_of_interest, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Areas of interest have been pickled and saved.")


No new feature drawn. Nothing to retrieve.


In [6]:
for name, coordinates in areas_of_interest.items():
    print(f"Area: {name}, Coordinates: {coordinates}")

Area: Enschede, Coordinates: [[[6.795044, 52.175827], [6.795044, 52.267317], [6.985588, 52.267317], [6.985588, 52.175827], [6.795044, 52.175827]]]
Area: Boekelo, Coordinates: [[[6.707668, 52.184142], [6.707668, 52.232951], [6.833324, 52.232951], [6.833324, 52.184142], [6.707668, 52.184142]]]


In [7]:
#begin maart to eind mei 
DateStart = "2022-04-01"
DateEnd = "2022-05-31"
# Get user input for the area
input_area_name = input("Enter the name of the area: ")

# Check if the area name exists in the dictionary and proceed
if input_area_name in areas_of_interest:
    coordinates = areas_of_interest[input_area_name]

    coll = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(ee.Geometry.Polygon(coordinates)) \
    .filterDate(DateStart, DateEnd) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)

    # Add your further processing of myCollection here
    print("Earth Engine query executed for the area:", input_area_name)
    listOfImages = coll.aggregate_array('system:index').getInfo()
    print('Number of images in the collection: ', len(listOfImages))

else:
    print(f"No coordinates found for the area '{input_area_name}'. Please enter a valid area name.")

Earth Engine query executed for the area: Boekelo
Number of images in the collection:  12


In [8]:
def calculate_centroid(coordinates):
    if not coordinates or not coordinates[0]:
        return None

    num_points = len(coordinates[0])
    sum_lat = sum([coord[1] for coord in coordinates[0]])  # Sum of all latitudes
    sum_lon = sum([coord[0] for coord in coordinates[0]])  # Sum of all longitudes

    return sum_lon / num_points, sum_lat / num_points  # Return centroid as (longitude, latitude)

centroid = calculate_centroid(coordinates)
print("Centroid (Longitude, Latitude):", centroid)


Centroid (Longitude, Latitude): (6.757930400000001, 52.2036656)


In [9]:
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')  # B8 is NIR and B4 is Red for Sentinel-2
    return image.addBands(ndvi)

#add ndvi value as a channel to each img of the collection
with_ndvi = coll.map(add_ndvi)

In [10]:
def ndvi_mask(image, ndvi_threshold = 0.3):
    ndvi = image.select('NDVI')
    vegetation_mask = ndvi.gt(ndvi_threshold)  # Greater than threshold
    return image.updateMask(vegetation_mask)

#create a new collection for the masked images
masked_collection = with_ndvi.map(ndvi_mask)
median_ndvi_image = masked_collection.median()

In [11]:
NDVI_THRESHOLD = 0.35

def ndvi_mask(image):
    ndvi = image.select('NDVI')
    vegetation_mask = ndvi.gt(NDVI_THRESHOLD)  # Greater than threshold
    return vegetation_mask

#server side request to compute ndvi based mask
masked_collection = with_ndvi.map(ndvi_mask)

aoi = ee.Geometry.Polygon(coordinates, None, False) #cords to polygon casting

# Create a composite image (e.g., median)
median_mask = masked_collection.first().clip(aoi)

vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'green']
}

# Add to map for visualization
Map.addLayer(median_mask, vis_params, 'Vegetation Mask')
Map.add_colorbar(vis_params, discrete=True, label = f'NDVI Threshold @ {NDVI_THRESHOLD}')

In [12]:
Map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [13]:
#todo
#use the median_NDVI_mask, to perform the spectral-temporal analysis, we want to create difference maps between the first and last img of the collection.
#find the pixels that disapeerd due to a decrease in ndvi, and create a mask for those.

In [18]:
# Assuming 'image_collection' is your initial ImageCollection
# and 'ndvi_mask' is your function to compute NDVI

# Step 1: Select the first and last images
first_image = coll.first()
last_image = coll.sort('system:time_start', False).first()

# Step 2: Calculate NDVI for both images
ndvi_first = first_image.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndvi_last = last_image.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Step 3: Create a difference map
ndvi_difference = ndvi_last.subtract(ndvi_first).clip(aoi)

# Step 4: Identify Decreased NDVI
# Pixels with negative values indicate a decrease in NDVI
decreased_ndvi = ndvi_difference.lt(0)  # lt(0) checks for negative values

# Step 5: Create a Mask for Decreased NDVI
# This binary mask will have 1s where NDVI decreased and 0s elsewhere
decreased_ndvi_mask = decreased_ndvi.updateMask(decreased_ndvi)

# Visualize or Export the results
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'red']  # Non-decreased as white, decreased as red
}

# Add to map for visualization
Map.addLayer(decreased_ndvi_mask, vis_params, 'Decreased NDVI Mask')

# Optional: Export the mask
# task = ee.batch.Export.image.toDrive(...)
# task.start()

In [17]:
Map

Map(bottom=86742.0, center=[52.12505765485123, 367.3127746582032], controls=(WidgetControl(options=['position'…